In [42]:
# Notebook to iterate through PGA & EUR tournaments and calculate percentage wins based each end of round finishing position
import os, sys, csv
import pandas as pd

tour_id = 'PGA'         # PGA / EUR
dir_curr = 'C:\\Users\\lukem\\Google Drive\\Thetaco\\golf-APERX-eval\\'
if tour_id=='PGA':
    dir_source = 'C:\\Users\\lukem\\Dropbox\\PC\\Desktop\\Big Data\\Golf Data\\PGA Tour\\Scorecards\\4RND-Data'
    dir_target = 'C:\\Users\\lukem\\Dropbox\\PC\\Desktop\\Big Data\\Golf Data\\PGA Tour\\Scorecards\\APERX\\'
if tour_id=='EUR':
    dir_source = 'C:\\Users\\lukem\\Dropbox\\PC\\Desktop\\Big Data\\Golf Data\\European Tour\\Scorecards\\4RND-Data'
    dir_target = 'C:\\Users\\lukem\\Dropbox\\PC\\Desktop\\Big Data\\Golf Data\\European Tour\\Scorecards\\APERX\\'

In [ ]:
#Parse APERX files & calculate APERX stats
os.chdir(dir_target)
aper1_df = pd.DataFrame([i for i in range(1,157)], columns=['aper1'])
aper2_df = pd.DataFrame([i for i in range(1,157)], columns=['aper2'])
aper3_df = pd.DataFrame([i for i in range(1,157)], columns=['aper3'])
aper1_df['# wins'] = [0]*len(aper1_df)
aper2_df['# wins'] = [0]*len(aper2_df)
aper3_df['# wins'] = [0]*len(aper3_df)
aper1_df['dates'] = ['']*len(aper1_df)
aper2_df['dates'] = ['']*len(aper2_df)
aper3_df['dates'] = ['']*len(aper3_df)
cnt=0
for file in os.listdir('.'):
    cnt+=1
    print(f'{cnt}) parsing {file} and updating APERX stats')
    df = pd.read_csv(file, engine='python')
    aper1, aper2, aper3 = df.loc[0,'APER1'], df.loc[0,'APER2'], df.loc[0,'APER3']
    aper1_df.loc[aper1-1, '# wins']+=1
    aper2_df.loc[aper2-1, '# wins']+=1
    aper3_df.loc[aper3-1, '# wins']+=1
    aper1_df.loc[aper1-1, 'dates']+=', '+'_'.join(file.split('_')[:3])
    aper2_df.loc[aper2-1, 'dates']+=', '+'_'.join(file.split('_')[:3])
    aper3_df.loc[aper3-1, 'dates']+=', '+'_'.join(file.split('_')[:3])
aper1_df['% wins'] = aper1_df['# wins']/cnt
aper2_df['% wins'] = aper2_df['# wins']/cnt
aper3_df['% wins'] = aper3_df['# wins']/cnt
aper1_df.to_csv(dir_curr+tour_id+' - APER1 stats.csv', index=False)
aper2_df.to_csv(dir_curr+tour_id+' - APER2 stats.csv', index=False)
aper3_df.to_csv(dir_curr+tour_id+' - APER3 stats.csv', index=False)

In [ ]:
#Parse tournaments, calculate APER for rounds 1-3, output results to target directory
r1_sort = [str(i) for i in range(18,0,-1)]
r1_sort = ['R1']+r1_sort
r1_bin = [True for i in range(len(r1_sort))]
r2_sort = [str(i) for i in range(36,0,-1)]
r2_sort = ['R2_tot', 'R2']+r2_sort
r2_bin = [True for i in range(len(r2_sort))]
r3_sort = [str(i) for i in range(54,0,-1)]
r3_sort = ['R3_tot', 'R3']+r3_sort
r3_bin = [True for i in range(len(r3_sort))]
cnt=0
os.chdir(dir_source)
for file in os.listdir('.'):
    if not file.endswith('.csv'): continue
    cnt+=1
    print(f'{cnt}) {file}')
    df = pd.read_csv(file, engine='python')
    if not df.loc[0,'Thru']=='#Courses = 1': continue
    df.drop(0, inplace=True)
    df.drop(df[df['Pos'].isin(['777','888'])].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    #APER1
    df.sort_values(by=r1_sort, ascending=r1_bin, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['APER1'] = df.index+1
    #APER2
    df['R2_tot'] = df['R1']+df['R2']
    df.sort_values(by=r2_sort, ascending=r2_bin, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['APER2'] = df.index+1
    #APER3
    df['R3_tot'] = df['R1']+df['R2']+df['R3']
    df.loc[df['Pos']==666, 'R3_tot']=666
    df.sort_values(by=r3_sort, ascending=r3_bin, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df['APER3'] = df.index+1
    #Output
    df.sort_values(by='Pos', ascending=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(dir_target+file[:-4]+' - APERX.csv', index=False)
